## Dart Lab Tutorial

#### Starting interactive cells
We will use the cell below as a header file essentially.  We will import all of the nessecary packages from this cell, and use them throughout the rest of the document. Throughout this document, we will be using code from previous cells, and gradually building off of it to add more features. 

In [38]:
import math
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import math
import ipywidgets as widgets
from pylab import *
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, Math, Latex
from scipy.integrate import quad
from scipy import integrate
from sympy import *
from IPython.display import HTML
import matplotlib.ticker as plticker
import sympy as sp
from scipy.integrate import quad
PI = np.pi

In [79]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

### Example: Estimating the Temperature Outside pt. 1
The cell below is going to be our first interactive cell.  In this cell, we are able to adjust the silders to change our temperature value, and also change the size of our icon.  The purpose of this is to introduce the idea of **interactive cells**, and show some of the basic features that they include.

In [98]:
def f(temp, y, icon_size):
    plt.axis([-4, 4, -.1, .2])
    plt.xlabel('Temperature')
    plt.ylabel('Probability')
    plt.title('An observation has a value (*),')
    plt.yticks([0, 0.2, 0.4, 0.6])
    plt.xticks([-4, -2, 0, 2, 4])
    plt.grid(True)
    plt.scatter([temp], [y], marker='*', s=icon_size, c='r')
    plt.show()
   
set_temp = widgets.FloatSlider(value=1.0, min=-4.0, max=4.0)
seticon_size = widgets.FloatSlider(value=300, min=10, max= 1000)

interact(f, temp=(set_temp), y=fixed(0), icon_size=(seticon_size))



interactive(children=(FloatSlider(value=1.0, description='temp', max=4.0, min=-4.0), FloatSlider(value=300.0, …

<function __main__.f(temp, y, icon_size)>

### Estimating the Temperature outside pt. 2
Here in the cell below, we have a similar graph as before, but added an error distribution (blue curve) that is associated with the instrument.   This **gaussian function** can be used as a basis for future graphs, and will be called upon multiple times in the rest of this document. Instrument builder says thermometer is unbiased with =/- 0.8°C gaussian error.

The red plot is $P(\frac{T}{T_0})$; probability of temperature *given* that $T_0$ was observed.

In [97]:
# Base gaussian function
def gaussian(obs, obs_error_SD):
    x=np.linspace(-10, 10, num=1000)
    y=1/(obs_error_SD*np.sqrt(2*np.pi))*np.exp((-1/2)*((x-obs)/obs_error_SD)**2)
    plt.plot(x,y, c='r')
    plt.scatter(obs, 0, c='r')
    plt.axis([-4, 4, -.1, .2])
    plt.xlabel('Temperature')
    plt.ylabel('Probability')
    plt.title('An observation has a value (*),')
    plt.yticks([0, 0.2, 0.4, 0.6])
    plt.xticks([-4, -2, 0, 2, 4])
    plt.grid(True)
    plt.show()

setobs = widgets.FloatSlider(value=1.0, min=-4.0, max=4.0)
obs_error_SD = widgets.FloatSlider(value=0.80, min=0.1, max=10)

interact(gaussian, obs=(setobs), obs_error_SD=(setobs_error_SD))
    

interactive(children=(FloatSlider(value=1.0, description='obs', max=4.0, min=-4.0), FloatSlider(value=0.8, des…

<function __main__.gaussian(obs, obs_error_SD)>

### We also have a prior estimate of temperature.
**Any future lines using the color green will be refering to prior PDF.**

In [104]:
# Base gaussian function
def gaussian(obs, obs_error_SD):
    x=np.linspace(-10, 10, num=1000)
    y=1/(obs_error_SD*np.sqrt(2*np.pi))*np.exp((-1/2)*((x-obs)/obs_error_SD)**2)
    plt.plot(x,y, c='g')
    plt.axis([-4, 4, -.1, .2])
    plt.xlabel('Temperature')
    plt.ylabel('Probability')
    plt.title('Prior Estimate of Temperature')
    plt.yticks([0, 0.2, 0.4, 0.6])
    plt.xticks([-4, -2, 0, 2, 4])
    plt.grid(True)
    plt.text(-3.5, .25, r'Prior PDF')
    plt.show()

setobs = widgets.FloatSlider(value=-1.0, min=-4.0, max=4.0)
setobs_error_SD = widgets.FloatSlider(value=1.30, min=0.1, max=10)   

interact(gaussian, obs=(setobs), obs_error_SD=(setobs_error_SD))
    

interactive(children=(FloatSlider(value=-1.0, description='obs', max=4.0, min=-4.0), FloatSlider(value=1.3, de…

<function __main__.gaussian(obs, obs_error_SD)>

The green curve is $P(\frac{T}{T_0})$;
probability of temperature given all available **prior** information $C$.

Prior information $C$ can include:

1. Observations of things besides $T$;
2. Model forecast made using observations at earlier times;
3. *A priori* physical constraints $(T > -273.15°C)$;
4. Climatological constraints $(-30°C < T < 40°C)$.

### Combining the Prior Estimate and Observation
#### Bayes Theorem: 
$$P(T|T_0,C)=\frac{P(T_0|T,C)P(T|C)}{P(T_0|C)}$$

The LHS represents the **Posterior**: Probability of $T$ given observations and Prior.  Also called **update** or **analysis**.

While the RHS represents the **likelihood**: Probability that $T_0$ is observed if $T$ is a true value and given prior information $C$.

Rewrite Bayes as:

$$\frac{P(T_0|T,C)P(T|C)}{P(T_0|C)}$$

$$\frac{P(T_0|T,C)P(T|C)}{\int P(T_0|x)P(x|C)dx}$$

$$\frac{\color{red}{P(T_0|T,C)}\color{green}{P(T|C)}}{normalization}$$

Denominator normalizes so Posterior is PDF.

In [105]:
# Base gaussian function
def gaussian(obs, obs_error_SD):
    x=np.linspace(-10, 10, num=1000)
    y=1/(obs_error_SD*np.sqrt(2*np.pi))*np.exp((-1/2)*((x-obs)/obs_error_SD)**2)
    plt.plot(x,y, c='g')
    plt.axis([-4, 4, -.1, .2])
    plt.xlabel('Temperature')
    plt.ylabel('Probability')
    plt.title('Prior Estimate of Temperature')
    plt.yticks([0, 0.2, 0.4, 0.6])
    plt.xticks([-4, -2, 0, 2, 4])
    plt.grid(True)
    plt.text(-3.5, .25, r'Prior PDF')
    plt.show()

setobs = widgets.FloatSlider(value=-1.0, min=-4.0, max=4.0)
setobs_error_SD = widgets.FloatSlider(value=1.30, min=0.1, max=10)   

interact(gaussian, obs=(setobs), obs_error_SD=(setobs_error_SD))
    

interactive(children=(FloatSlider(value=-1.0, description='obs', max=4.0, min=-4.0), FloatSlider(value=1.3, de…

<function __main__.gaussian(obs, obs_error_SD)>

In [106]:
def gaussian(x, mu, sigma):
    x=np.linspace(-10, 10, num=1000)
    y=1/(sigma*np.sqrt(2*np.pi))*np.exp((-1/2)*((x-mu)/sigma)**2)
    return y
    
def plot2gaussians(obs, obs_error_SD, prior_mean, prior_SD):
    x=np.linspace(-10, 10, num=1000)
    likelihood=gaussian(x, obs, obs_error_SD)
    prior=gaussian(x, prior_mean, prior_SD)
    
    plt.axis([-4, 4, -.1, .2])
    plt.xlabel('Temperature')
    plt.ylabel('Probability')
    plt.title('Prior Estimate and Observation')
    plt.text(2.1, .56, 'Obs. Likelihood')
    plt.yticks([0, 0.2, 0.4, 0.6])
    plt.xticks([-4, -2, 0, 2, 4])
    plt.grid(True)
    
    plt.plot(x, likelihood, c='r')
    plt.plot(x, prior, c='g')
    

    plt.show()

setobs = widgets.FloatSlider(value=1.0, min=-4.0, max=4.0)
setobs_error_SD = widgets.FloatSlider(value=0.80, min=0.1, max=10.0)
setprior_mean = widgets.FloatSlider(value=-1.0, min=-4.0, max=4.0)
setprior_SD = widgets.FloatSlider(value=1.10, min=0.1, max=10.0)

interact(plot2gaussians, obs=(setobs), obs_error_SD=(setobs_error_SD),\
         prior_mean=(setprior_mean), prior_SD=(setprior_SD) )

interactive(children=(FloatSlider(value=1.0, description='obs', max=4.0, min=-4.0), FloatSlider(value=0.8, des…

<function __main__.plot2gaussians(obs, obs_error_SD, prior_mean, prior_SD)>

$$P(T|T_0,C)=\frac{\color{red}{P(T_0|T,C)}\color{green}{P(T|C)}}{normalization}$$

In [107]:
def gaussian(x, mu, sigma):
    x=np.linspace(-10, 10, num=1000)
    y=1/(sigma*np.sqrt(2*np.pi))*np.exp((-1/2)*((x-mu)/sigma)**2)
    return y
    
def plot2gaussians(obs, obs_error_SD, prior_mean, prior_SD):
    x=np.linspace(-10, 10, num=1000)
    likelihood=gaussian(x, obs, obs_error_SD)
    prior=gaussian(x, prior_mean, prior_SD)
    numerator=(prior*likelihood)
    
    plt.axis([-4, 4, -.1, .2])
    plt.xlabel('Temperature')
    plt.ylabel('Probability')
    plt.title('Prior Estimate and Observation')
    plt.yticks([0, 0.2, 0.4, 0.6])
    plt.xticks([-4, -2, 0, 2, 4])
    plt.grid(True)
    
    plt.plot(x, likelihood, c='r')
    plt.plot(x, prior, c='g')
    plt.plot(x, numerator, c='k')
    
    plt.show()

setobs = widgets.FloatSlider(value=1.0, min=-4.0, max=4.0)
setobs_error_SD = widgets.FloatSlider(value=0.80, min=0.1, max=10.0)
setprior_mean = widgets.FloatSlider(value=-1.0, min=-4.0, max=4.0)
setprior_SD = widgets.FloatSlider(value=1.10, min=0.1, max=10.0)

interact(plot2gaussians, obs=(setobs), obs_error_SD=(setobs_error_SD),\
         prior_mean=(setprior_mean), prior_SD=(setprior_SD) )

interactive(children=(FloatSlider(value=1.0, description='obs', max=4.0, min=-4.0), FloatSlider(value=0.8, des…

<function __main__.plot2gaussians(obs, obs_error_SD, prior_mean, prior_SD)>

$$P(T|T_0,C)=\frac{\color{red}{P(T_0|T,C)}\color{green}{P(T|C)}}{\color{purple}{normalization}}$$

In [74]:
def gaussian(x, mu, sigma):
    x=np.linspace(-10, 10, num=1000)
    y=1/(sigma*np.sqrt(2*np.pi))*np.exp((-1/2)*((x-mu)/sigma)**2)
    return y
    
def plot2gaussians(obs, obs_error_SD, prior_mean, prior_SD):
    x=np.linspace(-10, 10, num=1000)
    likelihood=gaussian(x, obs, obs_error_SD)
    prior=gaussian(x, prior_mean, prior_SD)
    numerator=(prior*likelihood)
    
    plt.axis([-4, 4, -.1, .2])
    plt.xlabel('Temperature')
    plt.ylabel('Probability')
    plt.title('Prior Estimate and Observation')
    plt.text(-2.1, .56, 'Area Under Product is Denominator')
    plt.yticks([0, 0.2, 0.4, 0.6])
    plt.xticks([-4, -2, 0, 2, 4])
    plt.grid(True)
    
    plt.plot(x, likelihood, c='r')
    plt.plot(x, prior, c='g')
    plt.plot(x, numerator, c='k')
    fill_between(x, numerator, y2=0, facecolor='purple')
    
    plt.show()

setobs = widgets.FloatSlider(value=1.0, min=-4.0, max=4.0)
setobs_error_SD = widgets.FloatSlider(value=0.80, min=0.1, max=10.0)
setprior_mean = widgets.FloatSlider(value=-1.0, min=-4.0, max=4.0)
setprior_SD = widgets.FloatSlider(value=1.10, min=0.1, max=10.0)

interact(plot2gaussians, obs=(setobs), obs_error_SD=(setobs_error_SD),\
         prior_mean=(setprior_mean), prior_SD=(setprior_SD) )


interactive(children=(FloatSlider(value=1.0, description='obs', max=4.0, min=-4.0), FloatSlider(value=0.8, des…

<function __main__.plot2gaussians(obs, obs_error_SD, prior_mean, prior_SD)>

$$\color{blue}{P(T|T_0,C)}=\frac{\color{red}{P(T_0|T,C)}\color{green}{P(T|C)}}{\color{purple}{normalization}}$$

In [77]:
def gaussian(x, mu, sigma):
    x=np.linspace(-10, 10, num=1000)
    y=1/(sigma*np.sqrt(2*np.pi))*np.exp((-1/2)*((x-mu)/sigma)**2)
    return y
    
def plot2gaussians(obs, obs_error_SD, prior_mean, prior_SD):
    x=np.linspace(-10, 10, num=1000)
    likelihood=gaussian(x, obs, obs_error_SD)
    prior=gaussian(x, prior_mean, prior_SD)
    numerator=(prior*likelihood)
    weight=(1 / (np.sqrt(2 * np.pi) * np.sqrt(prior_SD**2 + obs_error_SD**2))\
            * np.exp(-0.5 * obs - prior_mean)**2 / ((prior_SD**2) + (obs_error_SD**2)))
    post_PDF=(numerator/weight)
    
    plt.axis([-4, 4, -.1, .2])
    plt.xlabel('Temperature')
    plt.ylabel('Probability')
    plt.title('Prior Estimate and Observation')
    plt.text(-2.1, .56, 'Posterior PDF')
    plt.yticks([0, 0.2, 0.4, 0.6])
    plt.xticks([-4, -2, 0, 2, 4])
    plt.grid(True)
    
    plt.plot(x, likelihood, c='r')
    plt.plot(x, prior, c='g')
    plt.plot(x, numerator, c='k')
    plt.plot(x, post_PDF, c='b')
    fill_between(x, numerator, y2=0, facecolor='purple')
    
    plt.show()

setobs = widgets.FloatSlider(value=1.0, min=-4.0, max=4.0)
setobs_error_SD = widgets.FloatSlider(value=0.80, min=0.1, max=10.0)
setprior_mean = widgets.FloatSlider(value=-1.0, min=-4.0, max=4.0)
setprior_SD = widgets.FloatSlider(value=1.10, min=0.1, max=10.0)

interact(plot2gaussians, obs=(setobs), obs_error_SD=(setobs_error_SD),\
         prior_mean=(setprior_mean), prior_SD=(setprior_SD) )

interactive(children=(FloatSlider(value=1.0, description='obs', max=4.0, min=-4.0), FloatSlider(value=0.8, des…

<function __main__.plot2gaussians(obs, obs_error_SD, prior_mean, prior_SD)>

### Consistent Color Scheme Throughout Tutorial
$$\color{green}{\textrm{Green}=\textrm{Prior}}$$
$$\color{red}{\textrm{Red}=\textrm{Observation}}$$
$$\color{blue}{\textrm{Blue}=\textrm{Posterior}}$$
$$\color{black}{\textrm{Black}=\textrm{Truth}}$$
$$\textrm{(truth available only for 'perfect model' examples)}$$
$$\color{blue}{P(T|T_0,C)}=\frac{\color{red}{P(T_0|T,C)}\color{green}{P(T|C)}}{\color{black}{normalization}}$$
$$\textrm{Generally no analytic solution for Posterior.}$$

In [137]:
def gaussian(x, mu, sigma):
    y=1/(sigma*np.sqrt(2*np.pi))*np.exp((-1/2)*((x-mu)/sigma)**2)
    return y
def product_of_gaussians(mean_1, SD_1, mean_2, SD_2):
    var_1=SD_1**2
    var_2=SD_2**2
    new_var = 1.0 / ((1.0 / var_1) + (1.0 / var_2))
    new_sd = np.sqrt(new_var)
    new_mean = new_var * ((mean_1 / var_1) + (mean_2 / var_2))
    return new_mean, new_sd
    
    
def plot2gaussians(obs, obs_error_SD, prior_mean, prior_SD):
    x=np.linspace(-4, 4, num=1000)
    likelihood=gaussian(x, obs, obs_error_SD)
    prior=gaussian(x, prior_mean, prior_SD)
    post_mean, post_sd = product_of_gaussians(prior_mean, prior_SD, obs, obs_error_SD)
    post=gaussian(x, post_mean, post_sd)
    numerator=(prior*likelihood)
    print(np.sum(prior,))
    print(np.sum(likelihood))
    print(np.sum(post))
    print(np.sum(numerator))
    weight=(1.0 / (np.sqrt(2.0 * np.pi) * np.sqrt(prior_SD**2 + obs_error_SD**2))\
            * np.exp(-0.5 * (obs - prior_mean))**2 / ((prior_SD**2) + (obs_error_SD**2)))
    post_PDF=(numerator/weight)
    print(np.sum(post_PDF))
    #plt.axis([-4, 4,])
    plt.xlabel('Temperature')
    plt.ylabel('Probability')
    plt.title('Prior Estimate and Observation')
    plt.text(-2.1, .56, 'Posterior PDF')
    #plt.yticks([0, 0.2, 0.4, 0.6])
    plt.xticks([-4, -2, 0, 2, 4])
    plt.grid(True)
    
    plt.plot(x, likelihood, c='r')
    plt.plot(x, prior, c='g')
    plt.plot(x, numerator, c='k')
    #plt.plot(x, post_PDF, c='b')
    plt.plot(x, post, c='b')
    fill_between(x, numerator, y2=0, facecolor='purple')
    
    plt.show()

setobs = widgets.FloatSlider(value=1.0, min=-4.0, max=4.0)
setobs_error_SD = widgets.FloatSlider(value=0.80, min=0.1, max=10.0)
setprior_mean = widgets.FloatSlider(value=-1.0, min=-4.0, max=4.0)
setprior_SD = widgets.FloatSlider(value=1.10, min=0.1, max=10.0)

interact(plot2gaussians, obs=(setobs), obs_error_SD=(setobs_error_SD),\
         prior_mean=(setprior_mean), prior_SD=(setprior_SD) )

interactive(children=(FloatSlider(value=1.0, description='obs', max=4.0, min=-4.0), FloatSlider(value=0.8, des…

<function __main__.plot2gaussians(obs, obs_error_SD, prior_mean, prior_SD)>

$$\color{blue}{P(T|T_0,C)}=\frac{\color{red}{P(T_0|T,C)}\color{green}{P(T|C)}}{\color{black}{normalization}}$$
$$\textrm{Gaussian Prior and Likelihood -> Gaussian Posterior}$$

In [121]:
def gaussian(x, mu, sigma):
    x=np.linspace(-10, 10, num=1000)
    y=1/(sigma*np.sqrt(2*np.pi))*np.exp((-1/2)*((x-mu)/sigma)**2)
    return y
    
def plot2gaussians(obs, obs_error_SD, prior_mean, prior_SD):
    x=np.linspace(-10, 10, num=1000)
    likelihood=gaussian(x, obs, obs_error_SD)
    prior=gaussian(x, prior_mean, prior_SD)
    numerator=(prior*likelihood)
    weight=(1 / (np.sqrt(2 * np.pi) * np.sqrt((prior_SD**2) + (obs_error_SD**2)))\
            * np.exp(-0.5 * (obs - prior_mean))**2 / ((prior_SD**2) + (obs_error_SD**2)))
    post_PDF=(numerator/weight)
    
    plt.axis([-4, 4, -.1, .2])
    plt.xlabel('Temperature')
    plt.ylabel('Probability')
    plt.title('Prior Estimate and Observation')
    plt.text(-2.1, .56, 'Posterior PDF')
    plt.yticks([0, 0.2, 0.4, 0.6])
    plt.xticks([-4, -2, 0, 2, 4])
    plt.grid(True)
    
    plt.plot(x, likelihood, c='r')
    plt.plot(x, prior, c='g')
    plt.plot(x, numerator, c='k')
    plt.plot(x, post_PDF, c='b')
    fill_between(x, numerator, y2=0, facecolor='purple')
    
    plt.show()

setobs = widgets.FloatSlider(value=1.0, min=-4.0, max=4.0)
setobs_error_SD = widgets.FloatSlider(value=0.80, min=0.1, max=10.0)
setprior_mean = widgets.FloatSlider(value=-1.0, min=-4.0, max=4.0)
setprior_SD = widgets.FloatSlider(value=1.10, min=0.1, max=10.0)

interact(plot2gaussians, obs=(setobs), obs_error_SD=(setobs_error_SD),\
         prior_mean=(setprior_mean), prior_SD=(setprior_SD) )

interactive(children=(FloatSlider(value=1.0, description='obs', max=4.0, min=-4.0), FloatSlider(value=0.8, des…

<function __main__.plot2gaussians(obs, obs_error_SD, prior_mean, prior_SD)>